In [8]:
import re
import os
import pickle
import numpy as np
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader

import torch
from transformers import BertTokenizer,BertModel

In [ ]:
from utils.MIND import MIND

In [ ]:
config.signal_length = 20
config.his_size = 10
train_path = config.path + 'MIND/MINDdemo_dev/'
a = MIND(config, train_path + 'news.tsv', train_path + 'behaviors.tsv')

In [ ]:
s1 = Partition_Sampler(a, num_replicas=3, rank=0)
s2 = Partition_Sampler(a, num_replicas=3, rank=1)
s3 = Partition_Sampler(a, num_replicas=3, rank=2)
len(s3)

In [ ]:
loader1 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, sampler=s1)
records1 = list(loader1)

In [ ]:
loader2 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, sampler=s2)
records2 = list(loader2)

In [1]:
from transformers import BertTokenizer

In [11]:
t = BertTokenizer.from_pretrained('bert-base-uncased')

In [18]:
1-t('I love you',return_tensors='pt').attention_mask

tensor([[0, 0, 0, 0, 0]])

In [9]:
b = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
b.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)